In [1]:
import utility as ut

OPENAI_API_KEY = ut.OPENAI_API_KEY
LANGSMITH_API_KEY = ut.LANGSMITH_API_KEY
MODEL_NAME = ut.MODEL_NAME
print(MODEL_NAME)

gpt-4o-mini


In [2]:
from langchain.agents import create_agent
agent = create_agent(
    model = MODEL_NAME,
    system_prompt = "You are a full stack comedian"
)

NO STREAMING INVOKE

In [3]:
result = agent.invoke(
    {
        "messages": [
            {
                "role":"user",
                "content":"Tell me a joke"
            }
        ]
    }
)

print(result["messages"][1].content)

Why don't scientists trust atoms? 

Because they make up everything!


# VALUES
this streaming mode is explained in previous examples 

In [4]:
#Stream = values
for step in agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Tell me sports joke"
            }
        ]
    }, 
    stream_mode = "values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Tell me sports joke
================================== Ai Message ==================================

Why do basketball players love donuts?

Because they can't resist a good dunk! 🍩🏀


# Messages
Messages stream data token by token - the lowest latency possible. this is perfect for interactive applications like chatbots

In [5]:
for token, metadata in agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "Write me a Family friendly poem"
            }
        ]
    }, 
    stream_mode = "messages"
):
    print(f"{token.content}", end="")

In a cozy little house, not too far away,  
Lived a family of four, full of love every day.  
With a mom who baked cookies, warm from the oven,  
And a dad who told stories, the kind you’re lovin’.  

There’s a sister named Lily, with a spirit so bright,  
She dances like daisies in the soft morning light.  
And a brother named Ben, always ready to play,  
With a smile that could chase all the gray clouds away.  

They have a fluffy dog, named Max, oh so fun,  
Who chases his tail in the warm summer sun.  
Together they picnic, under trees, green and tall,  
With sandwiches, laughter, and games for them all.  

In the evening, they gather, with blankets in tow,  
To watch stars in the sky, while the cool breezes blow.  
With stories and laughter, they share all their dreams,  
In this family of love, everything’s better, it seems.  

So here’s to the moments, both big and the small,  
In a family like this, there’s room for us all.  
With joy in our hearts, and kindness in hand,  
Toge

# TOOLS CAN STREAM TOO!
Streaming generally means delivering information to user before the final result is ready. there are many cases where this is useful

a `get_stream_writer` writer allows to easily stream `custom` data from sources you create

In [6]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer

def get_weather(city:str) -> str:
    """Get weather for a given city"""
    writer = get_stream_writer()
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city:{city}")
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=MODEL_NAME,
    tools=[get_weather],
)

for chunk in agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is the weather in London?"
            }
        ]
    }, 
    stream_mode = ["values", "custom"],
):
    print(chunk)

('values', {'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='8ddff6ca-603e-428a-8c2c-b170ad6769cb')]})
('values', {'messages': [HumanMessage(content='What is the weather in London?', additional_kwargs={}, response_metadata={}, id='8ddff6ca-603e-428a-8c2c-b170ad6769cb'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 51, 'total_tokens': 65, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-Cd9WOfjDmkiNVzg2qGJSfLCYBpg9o', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--8daa7a71-f163-43a7-a76f-43b396db4a1a-0', tool_calls=

# Try different modes 
modify the stream mode and the select to produce different results

In [7]:
for chunk in agent.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": "What is the weather in London?"
            }
        ]
    }, 
    stream_mode = ["values", "custom"],
):
    if chunk[0] == "custom":
        print(chunk[1])

Looking up data for city: London
Acquired data for city:London
